# Predicting speech tags 

Refer: `Frame Level Classification of Speech.doc` for the problem description

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import timeit

In [3]:
# Path to home
PATH = os.getcwd()

In [4]:
# Check the work space is enabled  with GPU.
import torch
cuda = torch.cuda.is_available()
print(cuda)

False


In [5]:
data_folder = os.path.join(PATH, 'data')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train = np.load('/content/drive/My Drive/Colab Notebooks/ntrain.npy', allow_pickle=True)
train_labels = np.load('/content/drive/My Drive/Colab Notebooks/ntrain_labels.npy', allow_pickle=True)

dev = np.load('/content/drive/My Drive/Colab Notebooks/nval.npy', allow_pickle=True)
dev_labels = np.load('/content/drive/My Drive/Colab Notebooks/nval_labels.npy', allow_pickle=True)

In [7]:
train.shape

NameError: ignored

In [ ]:
# # Using subset of complete data when modifying

# dev = dev[:20]
# dev_labels = dev_labels[:20]
# train = train[:10]
# train_labels = train_labels[:10]

In [8]:
from torch.utils.data import DataLoader, Dataset, TensorDataset

class TensorDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        assert len(x) == len(y)
        self._x = x
        self._y = y
    
    def __len__(self):
        return len(self._x)
      
    def __getitem__(self, index):
        x_item = self._x[index]
        return torch.FloatTensor(x_item), torch.FloatTensor(self._y[index])

In [8]:
train_dataset = TensorDataset(train, train_labels)

load_train = DataLoader(
    train_dataset,
    batch_size = 1,
    shuffle=False,
    pin_memory=True
)

dev_dataset = TensorDataset(dev, dev_labels)

load_valid = DataLoader(
    dev_dataset,
    batch_size = 1
)

In [9]:
import torch
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [10]:
# DEVICE = 'cpu'
DEVICE

device(type='cpu')

In [11]:
embedding_dim = 13
hidden_dim = 10
vocab_size = 346 # [0-347]
layers=4

def hidden_init():
    return (torch.rand(layers*2, 1, hidden_dim).to(DEVICE) ,
            torch.rand(layers*2, 1, hidden_dim).to(DEVICE))

In [12]:
class LSTM_model(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTM_model, self).__init__()
        self.vocab_size = 346 #vocab_size change to 346 ????
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim, num_layers=layers, dropout = 0.2, bidirectional = True).to(DEVICE)
        self.linear = torch.nn.Linear(hidden_dim*2, vocab_size)       # *2 applied if bidir = true
        self.softmax = torch.nn.functional.softmax
        
    def forward(self, encrypted):
        lstm_in = encrypted.transpose(0,1)

        lstm_out, lstm_hidden = self.lstm(lstm_in.float(), hidden_init())
        
        scores = self.linear(lstm_out)
        scores = scores.transpose(1, 2)

        return scores

model = LSTM_model(vocab_size, embedding_dim, hidden_dim)

In [13]:
model = model.to(DEVICE)

In [15]:
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/trained_model_Hw5_2.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [18]:
# Printing validation loss at regular intervals
validation_time = len(train) / 20

In [19]:
print(validation_time)

983.5


In [16]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [17]:
losses = []

class LSTM_Trainer():
    def __init__(self, model):
        self.model = model
        self.loss_fn = torch.nn.CrossEntropyLoss().to(DEVICE)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

    def get_loss(self, encrypted, original) :
        encrypted = encrypted.to(DEVICE).long()
        original = original.to(DEVICE).long()

        scores = self.model.forward(encrypted)
        original = original.transpose(0,1)
        original = original.long()

        loss = self.loss_fn(scores, original)  # <- Training loss
        return loss

    def train(self, num_epochs):
        accuracies, max_accuracy = [], 0
        best_valid_loss = 10   # V.high initialization

        with open(os.path.join(PATH, 'history.csv'),'w') as writer:
            for N in range(num_epochs):
                start_save_time = time.monotonic()
                print('Epoch: {}'.format(N))
                for i, (encrypted, original) in enumerate(load_train):  #dataset(num_examples):
                    self.optimizer.zero_grad()
  
                    loss = self.get_loss(encrypted, original)  # <- Training loss
                    loss.backward()

                    self.optimizer.step()

                # Validation
                    if i % validation_time == 0:

                        print('Validation:' + str(i))
                        validation_loss = []
                        for (val_encrypted, val_original) in load_valid:    #val dataset(num_examples):
                            val_loss = self.get_loss(val_encrypted, val_original) 
                      
                            validation_loss.append(val_loss.item())

                        avg_loss = sum(validation_loss) / len(validation_loss)
                        print('Training Loss: {:6.4f}'.format(loss.item()))
                        print('Validation Loss: {:6.4f}'.format(avg_loss))        
                        writer.write(str(N)+','+str(i)+','+str(loss.item())+','+str(avg_loss))
                        writer.write('\n')

                # Saving the model after an epoch
                model_saved = os.path.join(PATH, 'model_' + str(N+1) + '.sav')
                torch.save(self.model.state_dict(), '/content/drive/My Drive/Colab Notebooks/trained_model_Hw5_0.pt')
                torch.save(self.model.state_dict(), model_saved)
                print('Train Loss at end of epoch: {:6.4f}'.format(loss.item()))
                end_save_time = time.monotonic()
                save_min, save_sec = epoch_time(start_save_time, end_save_time)
                print(f'Epoch: {N+1:02} | Epoch Time: {save_min}m {save_sec}s\n')
                print("\n----------------------------------------------------------------------\n");

In [ ]:
trainer = LSTM_Trainer(model)

Working on GPU

In [ ]:
n_epochs = 5

In [ ]:
# Below call starts training models for required epochs

trainer.train(n_epochs)

Epoch: 0


RuntimeError: ignored

In [ ]:
# torch.save(model.state_dict(), '/content/drive/My Drive/Colab Notebooks/trained_model/trained_model_Hw5_0.pt')

The above module will train the output `model` using which the below predictions on test will be carried out

In [18]:
ntest = np.load('/content/drive/My Drive/Colab Notebooks/ntest.npy',allow_pickle=True)

In [19]:
from torch.utils.data import DataLoader, Dataset, TensorDataset

class TestDataset(Dataset):
    def __init__(self, x):
        super().__init__()
        # assert len(x) == len(y)
        self._x = x
    
    def __len__(self):
        return len(self._x)
      
    def __getitem__(self, index):
        x_item = self._x[index]
        return torch.FloatTensor(np.array(x_item))

In [20]:
test = np.load(os.path.join(data_folder, '/content/drive/My Drive/Colab Notebooks/ntest.npy'), allow_pickle=True)

test_dataset = TestDataset(ntest)

load_test = DataLoader(
    test_dataset,
    batch_size = 1,
    shuffle=False,
    pin_memory=True
)

In [21]:
def test_model(model, test_loader):
  with torch.no_grad():
    model.eval()
    pred = []

    for batch_idx, (data) in enumerate(test_loader):   
      data = data.to(DEVICE)
      outputs = model(data)

      _, predicted = torch.max(outputs.data, 1)
      pred.append(predicted.cpu().numpy()[0])

    return np.array(pred)

pred = test_model(model, load_test)

In [25]:
soft = torch.nn.Softmax(dim=0)

with open('/content/drive/My Drive/Colab Notebooks/hw1_submission.csv', 'w') as output:
    output.write('id,label')
    output_id = 0
    for encrypted in load_test:
        
        scores = model.forward(encrypted)

        soft_scores = soft(scores[0])      
        predictions = torch.max(soft_scores, 0)   
        for prediction in predictions:
            output.write(f"{output_id},{prediction}")
            output_id += 1
        #(output_id + ',' + prediction)

In [26]:
with open('/content/drive/My Drive/Colab Notebooks/labelssssssss.csv', 'w') as w:
    w.write('id,label\n')
    for i in range(len(pred)):
            w.write(str(i)+','+str(pred[i])+'\n')